In [ ]:
# default_exp dataloader

In [ ]:
# export
import logging
logging.disable(logging.CRITICAL)
import torch
import pandas as pd
from transformers import DistilBertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.distributed import DistributedSampler

In [ ]:
# hide
from ipywidgets import interact
import numpy as np

# Data 

> data exploration and construction of the Dataloader for the SemEval-2019 Task 3 dataset (contextual emotion detection in text)

## Data exploration
Let us first take a look at the preprocessed data using Pandas.

In [ ]:
# export
def open_data(path):
    """
    returns a Pandas DataFrame consisting of the SemEval data at path `path`  
    """
    return pd.read_csv(path, delimiter='\t', index_col='id')

In [ ]:
path = 'data/clean_train.txt'
df = open_data(path)
df.head()

,turn1,turn2,turn3,label
id,,,,
0,do not worry i am girl,hmm how do i know if you are,what ' s ur name ?,others
1,when did i ?,saw many times i think shame,no . i never saw you,angry
2,by,by google chrome,where you live,others
3,u r ridiculous,i might be ridiculous but i am telling the tru...,u little disgusting whore,angry
4,just for time pass,wt do u do 4 a living then,maybe,others


In [ ]:
df.shape

(30160, 4)

As you can see we are dealing with very unformal language, typos, and bad grammar. Moreoever, we are given an unbalanced dataset:

In [ ]:
df['label'].value_counts()

others    14948
angry      5506
sad        5463
happy      4243
Name: label, dtype: int64

## Data Transforms

We now tokenize the examples into features and create attention masks according to the (Distil)Bert standard. The following image from Jay Alammar's great blog post [A Visual Guide to Using BERT for the First Time](https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/) visualizes the tokenization step.

![Tokenization explained visually](./images/bert-distilbert-tokenization-2-token-ids.png)

In [ ]:
# export 
def transform_data(df, max_seq_len):
    """
    returns the padded input ids and attention masks according to the DistilBert tokenizer
    """
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    
    def tokenize_fct(turn):
        return tokenizer.encode(turn, add_special_tokens=True, max_length=max_seq_len)
    
    tokenized = df[['turn1','turn2','turn3']].applymap(tokenize_fct)
    padded = torch.tensor([[ids + [0]*(max_seq_len-len(ids)) for ids in idx] for idx in tokenized.values])
    attention_mask = torch.where(padded != 0, torch.ones_like(padded), torch.zeros_like(padded))
    return padded, attention_mask

In [ ]:
max_seq_len = 10
padded, attention_mask = transform_data(df, max_seq_len)
assert padded.shape == attention_mask.shape == df[['turn1','turn2','turn3']].shape + (max_seq_len,)

Lets digest the outcome for the first two conversations.

In [ ]:
df[:2]

,turn1,turn2,turn3,label
id,,,,
0,do not worry i am girl,hmm how do i know if you are,what ' s ur name ?,others
1,when did i ?,saw many times i think shame,no . i never saw you,angry


The utterances are transformed according to the tokenizer vocabulary

In [ ]:
vocab = np.array(list(DistilBertTokenizer.from_pretrained('distilbert-base-uncased').vocab.items()))
vocab[padded[:2]].reshape(2,3,-1)

array([[['[CLS]', '101', 'do', '2079', 'not', '2025', 'worry', '4737',
         'i', '1045', 'am', '2572', 'girl', '2611', '[SEP]', '102',
         '[PAD]', '0', '[PAD]', '0'],
        ['[CLS]', '101', 'hmm', '17012', 'how', '2129', 'do', '2079',
         'i', '1045', 'know', '2113', 'if', '2065', 'you', '2017',
         'are', '2024', '[SEP]', '102'],
        ['[CLS]', '101', 'what', '2054', "'", '1005', 's', '1055', 'ur',
         '24471', 'name', '2171', '?', '1029', '[SEP]', '102', '[PAD]',
         '0', '[PAD]', '0']],

       [['[CLS]', '101', 'when', '2043', 'did', '2106', 'i', '1045',
         '?', '1029', '[SEP]', '102', '[PAD]', '0', '[PAD]', '0',
         '[PAD]', '0', '[PAD]', '0'],
        ['[CLS]', '101', 'saw', '2387', 'many', '2116', 'times', '2335',
         'i', '1045', 'think', '2228', 'shame', '9467', '[SEP]', '102',
         '[PAD]', '0', '[PAD]', '0'],
        ['[CLS]', '101', 'no', '2053', '.', '1012', 'i', '1045',
         'never', '2196', 'saw', '2387', 'you', 

...resulting in the corresponding input ids (padded with zeros to `max_seq_len`)

In [ ]:
padded[:2]

tensor([[[  101,  2079,  2025,  4737,  1045,  2572,  2611,   102,     0,     0],
         [  101, 17012,  2129,  2079,  1045,  2113,  2065,  2017,  2024,   102],
         [  101,  2054,  1005,  1055, 24471,  2171,  1029,   102,     0,     0]],

        [[  101,  2043,  2106,  1045,  1029,   102,     0,     0,     0,     0],
         [  101,  2387,  2116,  2335,  1045,  2228,  9467,   102,     0,     0],
         [  101,  2053,  1012,  1045,  2196,  2387,  2017,   102,     0,     0]]])

...and masks for the self-attention layers specifying the length of each utterance:

In [ ]:
attention_mask[:2]

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]],

        [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]])

We also transform the labels to integers according to a given dictionary `emo_dict`.

In [ ]:
# export 
def get_labels(df, emo_dict):
    """
    returns the labels according to the emotion dictionary
    """
    return torch.tensor([emo_dict[label] for label in df['label'].values])

In [ ]:
emo_dict = {'others': 0, 'sad': 1, 'angry': 2, 'happy': 3}
labels = get_labels(df, emo_dict)

Let us look at the result for our two conversations from above:

In [ ]:
labels[:2]

tensor([0, 2])

The following widget allows to interactively explore the former data transformations.

In [ ]:
# hide
def print_features(idx):
  input_ids = padded[idx,:].numpy()
  print('original Data:', df.values[idx,:3].reshape(-1,1),         
        'vocabulary:', vocab[input_ids].reshape(1,3,-1),
        'padded input ids:', input_ids,
        'attention mask:', attention_mask[idx,:].numpy(),
        'original label:', df.values[idx,3],
        'transformed label:', labels[idx].numpy(), sep = '\n\n')

interact(print_features, idx = (0,5));

interactive(children=(IntSlider(value=2, description='idx', max=5), Output()), _dom_classes=('widget-interact'…

## PyTorch Dataloader

Finally we aggregate all the functions above into a PyTorch dataloader (with optional [distributed training](https://pytorch.org/tutorials/beginner/aws_distributed_training_tutorial.html?highlight=distributedsampler)).

In [ ]:
# export 
def dataloader(path, max_seq_len, batch_size, emo_dict, use_ddp = False, labels = True):
    """
    Transforms the .csv data stored in `path` according to DistilBert features and returns it as a DataLoader 
    """
    df = open_data(path)
    padded, attention_mask = transform_data(df, max_seq_len)
    
    if labels:
        dataset = TensorDataset(padded, attention_mask, get_labels(df, emo_dict))
    
    else:
        dataset = TensorDataset(padded, attention_mask)
        
    if use_ddp:
        train_sampler = DistributedSampler(dataset)
    
    else: 
        train_sampler = None

    return DataLoader(dataset, batch_size=batch_size, shuffle=(train_sampler is None), 
                      sampler=train_sampler, num_workers=0)

In [ ]:
batch_size = 5
loader = dataloader(path, max_seq_len, batch_size, emo_dict)

A batch consists of `batch_size` input ids, attention_masks, and optionally the corresponding labels. 

In [ ]:
batch = next(iter(loader))
batch

[tensor([[[  101,  4283,   102,     0,     0,     0,     0,     0,     0,     0],
          [  101,  2017,  2024,  6160,   999,  2001,  2061,  2307,  1012,   102],
          [  101,  3861,  4604,  2033,   102,     0,     0,     0,     0,     0]],
 
         [[  101,  2748, 28516,  2024,  5754,  7685,  2075,   102,     0,     0],
          [  101,  3524,  2003,  2009,  1037, 28516,  2030,  2025,  1029,   102],
          [  101, 15703,   102,     0,     0,     0,     0,     0,     0,     0]],
 
         [[  101,  1045,  2064,  2025,  2066,  3422,   102,     0,     0,     0],
          [  101,  2323,  2025,  3422,  2021,  3666,   102,     0,     0,     0],
          [  101,  7929,   102,     0,     0,     0,     0,     0,     0,     0]],
 
         [[  101,  1045,  2572,  2025,   102,     0,     0,     0,     0,     0],
          [  101,  6343,  2056,  1057,  2020,   102,     0,     0,     0,     0],
          [  101,  1045,  2113,  1045,  2572,  3374,  3241,   102,     0,     0]],
 
    

We can find, for instance, the first conversation of this batch in our input ids above.

In [ ]:
conversation = batch[0][0]
assert torch.any(torch.all(torch.all(conversation == padded, dim=2), dim=1)) == True